In [1]:
import numpy as np
import pandas as pd
import os
import sys
import sqlite3
from datetime import datetime
from tqdm import tqdm


import PivotTable as pt
from process_mangement import TableTransformer as tt

In [2]:
process = "2201"
today= datetime.today().strftime('%Y%m%d')

In [ ]:
# 폴더경로
folder_base = os.getcwd()
folder_common = os.path.join(folder_base, "working_common")
folder_schema = os.path.join(folder_base, "schema")
folder_src = os.path.join(folder_base, "working_by_source")
folder_result = os.path.join(folder_base, "result")

# 분류체계 파일 로드
f_cct = "분류체계.csv"
path_cct = os.path.join(folder_schema, f_cct)
df_cct = pd.read_csv(path_cct)

# 속성체계 파일 로드
f_indiv = "속성체계.csv"
path_indiv = os.path.join(folder_schema, f_indiv)
df_indiv = pd.read_csv(path_indiv)

# 작업파일 로드
f_nm = "표준데이터시트-고정.csv"
f_path = os.path.join(folder_src, f_nm)
df_attrs_sta = pd.read_csv(f_path, encoding='utf8')

# 해더 파일 로드
header_path = os.path.join(folder_schema, "info_headers.csv")
df_headers = pd.read_csv(header_path, encoding='utf8')
df_headers = df_headers.fillna(value=np.nan)

In [ ]:
db_path = os.path.join(folder_base, "db", "db.sqlite3")
conn = sqlite3.connect(db_path)
cur = conn.cursor()

In [ ]:
table_name = "공통속성"
columns = [] # 업로드할 컬럼들을 입력
primary_key_columns = ['공정', 'SRNo']
primary_key_str = ', '.join(primary_key_columns)
update_columns_str = ', '.join([f"{col} = excluded.{col}" for col in columns if col not in primary_key_columns])

query = f"""
    INSERT INTO {table_name} ({', '.join(columns)})
    SELECT {', '.join(columns)} FROM temp_table
    ON CONFLICT ({primary_key_str}) DO UPDATE SET
    {update_columns_str}
"""

In [ ]:
# 임시 테이블 생성 및 데이터 삽입
df_attrs_sta.to_sql('temp_table', conn, if_exists='replace', index=False)

# 기존 테이블에 데이터 병합
cur.execute(query)

# 임시 테이블 삭제
cur.execute("DROP TABLE temp_table")

# 커밋 및 연결 종료
conn.commit()
conn.close()